In [1]:
#Import the neccesary libraries
import numpy as np
import argparse
import cv2 
from imutils.video import FPS
import pyttsx3
import speech_recognition as sr

# construct the argument parse 
#parser = argparse.ArgumentParser(description='Script to run MobileNet-SSD object detection network ')
#parser.add_argument("--video", help="path to video file. If empty, camera's stream will be used")
#parser.add_argument("--prototxt", default="MobileNetSSD_deploy.prototxt", help='Path to text network file: ' 'MobileNetSSD_deploy.prototxt for Caffe model or ')
#parser.add_argument("--weights", default="MobileNetSSD_deploy.caffemodel", help='Path to weights: ' 'MobileNetSSD_deploy.caffemodel for Caffe model or ' )
#parser.add_argument("--thr", default=0.2, type=float, help="confidence threshold to filter out weak detections")
#args = parser.parse_args()

# Labels of Network.
classNames = { 0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat', 5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair',
    10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse', 14: 'motorbike', 15: 'person', 16: 'pottedplant', 17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor' }

# Open video file or capture device. 
#url="http://192.168.0.102:8080/video"
#cap = cv2.VideoCapture(url)
cap = cv2.VideoCapture("sofa1.mp4")
#cap = cv2.VideoCapture(0)
fps = FPS().start()

en=pyttsx3.init()
prev_distance = 1000

mode = "none"
#Load the Caffe model 
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt", "MobileNetSSD_deploy.caffemodel")

        

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_resized = cv2.resize(frame,(300,300)) # resize frame for prediction

    # MobileNet requires fixed dimensions for input image(s)
    # so we have to ensure that it is resized to 300x300 pixels.
    # set a scale factor to image because network the objects has differents size. 
    # We perform a mean subtraction (127.5, 127.5, 127.5) to normalize the input;
    # after executing this command our "blob" now has the shape:
    # (1, 3, 300, 300)
    blob = cv2.dnn.blobFromImage(frame_resized, 0.007843, (300, 300), (127.5, 127.5, 127.5), False)
    #Set to network the input blob 
    net.setInput(blob)
    #Prediction of network
    detections = net.forward()

    #Size of frame resize (300x300)
    cols = frame_resized.shape[1] 
    rows = frame_resized.shape[0]

    
    F = 210
    D = 36
    W = 22
    
    #For get the class and location of object detected, 
    # There is a fix index for class, location and confidence
    # value in @detections array .
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2] #Confidence of prediction 
        if confidence > 0.8: # Filter prediction 
            class_id = int(detections[0, 0, i, 1]) # Class label
            
            if class_id == 9:
            
                # Object location 
                xLeftBottom = int(detections[0, 0, i, 3] * cols) 
                yLeftBottom = int(detections[0, 0, i, 4] * rows)
                xRightTop   = int(detections[0, 0, i, 5] * cols)
                yRightTop   = int(detections[0, 0, i, 6] * rows)
                
                width = round(xRightTop - xLeftBottom, 4)
                #print(width)
                #F = width*D/W
                #print(F)
                
                # Distance from camera based on triangle similarity
                distance = (22 * F)/width
                distance = distance/12
                #print("Distance(ft):{dist}\n".format(dist=distance))
                print("Distance is {} steps".format(int(distance)))

                # Factor for scale to original size of frame
                heightFactor = frame.shape[0]/300.0  
                widthFactor = frame.shape[1]/300.0 
                # Scale object detection to frame
                xLeftBottom = int(widthFactor * xLeftBottom) 
                yLeftBottom = int(heightFactor * yLeftBottom)
                xRightTop   = int(widthFactor * xRightTop)
                yRightTop   = int(heightFactor * yRightTop)
                # Draw location of object  
                cv2.rectangle(frame, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop), (0, 255, 0))

                # Draw label and confidence of prediction in frame resized
                if class_id in classNames:
                    label = classNames[class_id] + ": " + str(confidence)
                    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

                    yLeftBottom = max(yLeftBottom, labelSize[1])
                    cv2.rectangle(frame, (xLeftBottom, yLeftBottom - labelSize[1]), (xLeftBottom + labelSize[0], yLeftBottom + baseLine),(255, 255, 255), cv2.FILLED)
                    cv2.putText(frame, label, (xLeftBottom, yLeftBottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(label) #print class and confidence
            
            elif class_id == 18:
                
                # Object location 
                xLeftBottom = int(detections[0, 0, i, 3] * cols) 
                yLeftBottom = int(detections[0, 0, i, 4] * rows)
                xRightTop   = int(detections[0, 0, i, 5] * cols)
                yRightTop   = int(detections[0, 0, i, 6] * rows)
                
                width = round(xRightTop - xLeftBottom, 4)
                #print(width)
                #F = width*D/W
                #print(F)
                
                # Distance from camera based on triangle similarity
                distance = (84 * F)/width
                distance = distance/12
                distance = int(distance)
                #print("Distance(in):{dist}\n".format(dist=distance))
                print("Distance is {} steps".format(distance))
                if distance < prev_distance - 2:
                    en.say("Distance is {} steps".format(distance))
                    en.runAndWait() 
                    prev_distance = distance

                # Factor for scale to original size of frame
                heightFactor = frame.shape[0]/300.0  
                widthFactor = frame.shape[1]/300.0 
                # Scale object detection to frame
                xLeftBottom = int(widthFactor * xLeftBottom) 
                yLeftBottom = int(heightFactor * yLeftBottom)
                xRightTop   = int(widthFactor * xRightTop)
                yRightTop   = int(heightFactor * yRightTop)
                # Draw location of object  
                cv2.rectangle(frame, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop), (0, 255, 0))

                # Draw label and confidence of prediction in frame resized
                if class_id in classNames:
                    label = classNames[class_id] + ": " + str(confidence)
                    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

                    yLeftBottom = max(yLeftBottom, labelSize[1])
                    cv2.rectangle(frame, (xLeftBottom, yLeftBottom - labelSize[1]), (xLeftBottom + labelSize[0], yLeftBottom + baseLine),(255, 255, 255), cv2.FILLED)
                    cv2.putText(frame, label, (xLeftBottom, yLeftBottom), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                    print(label) #print class and confidence
        fps.update()
    
    fps.stop()
    #print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
    cv2.imshow("frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break


Distance is 9 steps
sofa: 0.8630589
Distance is 9 steps
sofa: 0.8100996
Distance is 9 steps
sofa: 0.95227784
Distance is 9 steps
sofa: 0.931901
Distance is 9 steps
sofa: 0.8194244
Distance is 8 steps
sofa: 0.9311025
Distance is 7 steps
sofa: 0.85546374
Distance is 6 steps
sofa: 0.9743173
Distance is 7 steps
sofa: 0.812533
Distance is 7 steps
sofa: 0.9696714
Distance is 7 steps
sofa: 0.85982496
Distance is 7 steps
sofa: 0.9755586
Distance is 7 steps
sofa: 0.9100618
Distance is 6 steps
sofa: 0.94890845
Distance is 6 steps
sofa: 0.97956675
Distance is 6 steps
sofa: 0.9785226
Distance is 6 steps
sofa: 0.99481565
Distance is 6 steps
sofa: 0.9955908
Distance is 6 steps
sofa: 0.99517953
Distance is 6 steps
sofa: 0.9972811
Distance is 6 steps
sofa: 0.9951255
Distance is 6 steps
sofa: 0.99902034
Distance is 6 steps
sofa: 0.9893606
Distance is 6 steps
sofa: 0.99676824
Distance is 5 steps
sofa: 0.99911684
Distance is 5 steps
sofa: 0.9938356
Distance is 5 steps
sofa: 0.996947
Distance is 5 steps
s

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
